# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This lists are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our purposes.

## Data

In [ ]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [ ]:
# since I'm working from google colab it's more comfortable to load files from
# google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
uk_emb_path = "/content/drive/MyDrive/Colab Notebooks/girafe-ai assignments/Word vectors/cc.uk.300.vec"
uk_emb = KeyedVectors.load_word2vec_format(uk_emb_path)

CPU times: user 10min 1s, sys: 7.89 s, total: 10min 9s
Wall time: 10min 12s


In [ ]:
%%time
ru_emb_path = "/content/drive/MyDrive/Colab Notebooks/girafe-ai assignments/Word vectors/cc.ru.300.vec"
ru_emb = KeyedVectors.load_word2vec_format(ru_emb_path)

CPU times: user 9min 59s, sys: 8.03 s, total: 10min 7s
Wall time: 10min 8s


In [ ]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [ ]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [ ]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [ ]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [ ]:
# !wget -O ukr_rus.train.txt http://tiny.cc/jfgecz
# Wrong link giving error 404. I loaded this .txt files manually

In [ ]:
# !wget -O ukr_rus.test.txt http://tiny.cc/6zoeez
# This link was ok but for better reliability i loaded this one manually too

In [ ]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [ ]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [ ]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
# mapping = ...
# -------

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [ ]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531432747840881),
 ('июнь', 0.8402522802352905),
 ('март', 0.8385884165763855),
 ('сентябрь', 0.8331484794616699),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278019428253174),
 ('ноябрь', 0.8243728280067444),
 ('июль', 0.8229618072509766),
 ('август', 0.8112280368804932),
 ('январь', 0.8022986650466919)]

We can see that neighbourhood of this embedding consists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [ ]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        word_metrcis_list = ru_emb.most_similar(
            mapped_vectors[i].reshape(1, -1),
            topn=topn
        )
        word_list = [word for word, metric in word_metrcis_list]
        if ru in word_list:
            num_matches += 1

    precision_val = num_matches / len(pairs)
    return precision_val

In [ ]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [ ]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [ ]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [ ]:
print(precision_top1)
print(precision_top5)

0.628498727735369
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [ ]:
import numpy as np

In [ ]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
    # compute orthogonal embedding space mapping
    # mapping = ...
    u_matrix, _, vt_matrix = np.linalg.svd(X_train.T @ Y_train)
    mapping = u_matrix @ vt_matrix

    return mapping

In [ ]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.805662989616394),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938188910484314),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

In [ ]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [ ]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz

--2021-10-18 17:18:37--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819128 (1.7M) [application/gzip]
Saving to: ‘uk.txt.gz’

uk.txt.gz           100%[===================>]   1.73M  7.80MB/s    in 0.2s    

2021-10-18 17:18:38 (7.80 MB/s) - ‘uk.txt.gz’ saved [1819128/1819128]



In [ ]:
!gzip -d ./uk.txt.gz

In [ ]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [ ]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

Если разделять слова по пробелам, то будут возникать случаи, когда "словом" будет считаться его написание слитно с предшествующим/последующим знаком препинания. При этом эмбеддинги известны для слов и знаков препинания отдельно. Чтобы на примерах типа "коледж," функция translate давала результат не "UNK", а "колледж,", требуется разделить "коледж," на "коледж" и ",". Потом перевести их, а затем склеить обратно. Для этого написаны две функции ниже.

In [ ]:
# Any necessary preprocessing if needed
# YOUR CODE HERE

# знаки пунктуации, пищущиеся со словами слитно
punct_signs = ['.', '.\n', ',', ',\n', '?', '?\n', '!', '!\n', '»', '»\n',
               ';', ';\n', '«']


def split_punct(sentence):
    changed_sent = list()
    for word in sentence.split(" "):
        changed_word = list()
        for c in word:
            if c in punct_signs[:-1]:
                changed_word.append(" ")
                changed_word.append(c)
            elif c == punct_signs[-1]:
                changed_word.append(c)
                changed_word.append(" ")
            else:
                changed_word.append(c)

        changed_sent.append("".join(changed_word))

    return " ".join(changed_sent)


def unsplit_punct(sentence):
    changed_sent = list()
    for word in sentence.split(" "):
        if word in punct_signs[:-1] and len(changed_sent) > 0:
            changed_sent[-1] += word
        elif len(changed_sent) > 0 and changed_sent[-1] == punct_signs[-1]:
            changed_sent[-1] += word
        else:
            changed_sent.append(word)
        
    return " ".join(changed_sent)

test_sent = "«Скільки коштує ця носова хусточка?» — «Дев'яносто п'ять центів».\n"
print("Original sentence: " + test_sent)

splitted_test_sent = split_punct(test_sent)
print("Splitted sentence: " + splitted_test_sent)

unsplitted_test_sent = unsplit_punct(splitted_test_sent)
print("Unsplitted sentence: " + unsplitted_test_sent)

Original sentence: «Скільки коштує ця носова хусточка?» — «Дев'яносто п'ять центів».

Splitted sentence: « Скільки коштує ця носова хусточка ? » — « Дев'яносто п'ять центів » .

Unsplitted sentence: «Скільки коштує ця носова хусточка?» — «Дев'яносто п'ять центів».



In [ ]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
    translated = []
    sentence = split_punct(sentence)
    for uk_word in sentence.split(" "):
        # no need for translating empty strings
        if uk_word == '':
            continue

        # There are no embeddings for words that contain
        # ' and  ’ in uk_emb. Instead there are embeddings
        # for words without these symbols. For example,
        # uk_emb has no idea about "здоров'я" but knows
        # "здоровя". So we fix this by 2 following lines.
        uk_word = uk_word.replace('\'', '')
        uk_word = uk_word.replace('’', '')

        # try to find embedding in uk_emb for current uk_word
        try:
            translated.append(
                ru_emb.most_similar(
                    [np.matmul(uk_emb[uk_word], W)],
                    topn=1
                )[0][0]
            )

        except KeyError:
            # This one fixes situations such as '.\n', '?\n'
            # and so on. Since these expressions are equal in both
            # russian and ukranian we can just use uk_word as translation
            if uk_word in punct_signs[:-1]:
                translated.append(uk_word)

            else:
                # Maybe there is embedding available for lower-case word?
                uk_word = uk_word.lower()
                try:
                    translated.append(
                        ru_emb.most_similar(
                            [np.matmul(uk_emb[uk_word], W)],
                            topn=1
                        )[0][0]
                    )

                except KeyError:
                    translated.append("UNK")

    translated = " ".join(translated)
    return unsplit_punct(translated)

In [ ]:
assert translate(".") == "."
assert translate("1, 3") == "1, 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [ ]:
for ind, sent in enumerate(uk_corpus[::10]):
    i = ind + 1
    if i < 10:
        sep = " " * 3
    elif i < 100:
        sep = " " * 4
    elif i < 1000:
        sep = " " * 5

    print(f"{i}. Перевод: " + translate(sent), "Оригинал: " + sent, sep=sep)

1. Перевод: Я уже закончу колледж, когда мы прибежишь со Америки.
   Оригинал: Я вже закінчу коледж, коли ви вернетеся з Америки.

2. Перевод: Город бомбили враждебные самолеты.
   Оригинал: Місто бомбардували ворожі літаки.

3. Перевод: Возможно, мной антисоциальный, конечно это не означает, что мной не общаюсь со людьми.
   Оригинал: Можливо, я антисоціальний, але це не означає, що я не спілкуюся з людьми.

4. Перевод: Впрочем утра выпала роса.
   Оригинал: Цього ранку випала роса.

5. Перевод: Беда не приходит одна.
   Оригинал: Біда не приходить одна.

6. Перевод: Посмотри по тот дым.
   Оригинал: Подивися на той дим.

7. Перевод: Я заказал два гамбургера.
   Оригинал: Я замовив два гамбургера.

8. Перевод: Я не хотел никого обидеть.
   Оригинал: Я не хотів нікого образити.

9. Перевод: Гора покрыта снегом.
   Оригинал: Гора вкрита снігом.

10. Перевод: по фотографии во девушки корона не со золота, а со цветов.
    Оригинал: На фотографії в дівчини корона не з золота, а з квітів.



В некоторых местах появляются UNK, что связано с единичными опечатками (например, "Амазонка—" вместо "Амазонка —"). Тем не менее, теперь такие ситуации сведены к минимуму.

Great! 
See second notebook for the Neural Machine Translation assignment.